<a href="https://colab.research.google.com/github/jperdomos/Procesamiento-digital-de-se-ales/blob/main/Se%C3%B1alChirp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipywidgets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.5 MB/s eta 0:00:00


In [ ]:

from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import chirp, spectrogram
from scipy.fft import fft, fftfreq
import ipywidgets as widgets
from ipywidgets import interact


In [ ]:
def analizar_chirp(f0=5, f1=50, T=2.0, ventana_ini=0.0, ventana_dur=0.5):
    fs = 1000
    t = np.linspace(0, T, int(T*fs), endpoint=False)
    x = chirp(t, f0=f0, f1=f1, t1=T, method='linear')

    # Ventana de análisis
    ini_idx = int(ventana_ini * fs)
    dur_idx = int(ventana_dur * fs)
    fin_idx = ini_idx + dur_idx
    ventana_t = t[ini_idx:fin_idx]
    ventana_x = x[ini_idx:fin_idx]

    # FFT de la ventana
    N = len(ventana_x)
    freqs = fftfreq(N, d=1/fs)
    X = fft(ventana_x)

    # Gráficas
    plt.figure(figsize=(16, 8))

    # Señal completa
    plt.subplot(2, 2, 1)
    plt.plot(t, x, label='Señal chirp')
    plt.axvspan(ventana_ini, ventana_ini + ventana_dur, color='red', alpha=0.3, label='Ventana')
    plt.title("Señal chirp con ventana de análisis")
    plt.xlabel("Tiempo [s]")
    plt.ylabel("Amplitud")
    plt.grid(True)
    plt.legend()

    # Señal de la ventana
    plt.subplot(2, 2, 2)
    plt.plot(ventana_t, ventana_x)
    plt.title("Señal en la ventana")
    plt.xlabel("Tiempo [s]")
    plt.grid(True)

    # Magnitud de la FFT
    plt.subplot(2, 2, 3)
    plt.stem(freqs[:N//2], np.abs(X[:N//2]), basefmt=" ")
    plt.xlim(0, np.max(freqs[:N//2]))
    plt.xticks(np.arange(0, np.max(freqs[:N//2]), step=10))
    plt.title("Espectro (DFT) de la ventana")
    plt.xlabel("Frecuencia [Hz]")
    plt.ylabel("Magnitud")
    plt.grid(True)


    # Espectrograma
    plt.subplot(2, 2, 4)
    f, tt, Sxx = spectrogram(x, fs=fs, nperseg=128, noverlap=64)
    plt.pcolormesh(tt, f, Sxx, shading='gouraud')
    plt.title("Espectrograma de la señal")
    plt.ylabel("Frecuencia [Hz]")
    plt.xlabel("Tiempo [s]")
    plt.colorbar(label="Potencia")

    plt.tight_layout()
    plt.show()


In [ ]:
interact(
    analizar_chirp,
    f0=widgets.FloatSlider(value=5, min=1, max=50, step=1, description="f0 (Hz)"),
    f1=widgets.FloatSlider(value=50, min=10, max=200, step=1, description="f1 (Hz)"),
    T=widgets.FloatSlider(value=2.0, min=1.0, max=20.0, step=0.1, description="Duración (s)"),
    ventana_ini=widgets.FloatSlider(value=0.0, min=0.0, max=20.0, step=0.1, description="Inicio ventana (s)"),
    ventana_dur=widgets.FloatSlider(value=0.5, min=0.1, max=20.0, step=0.1, description="Duración ventana (s)")
)


interactive(children=(FloatSlider(value=5.0, description='f0 (Hz)', max=50.0, min=1.0, step=1.0), FloatSlider(…

<function __main__.analizar_chirp(f0=5, f1=50, T=2.0, ventana_ini=0.0, ventana_dur=0.5)>

In [ ]:
!pip install ipywidgets matplotlib scipy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.1 MB/s eta 0:00:00


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, sosfilt
from ipywidgets import interact, IntSlider, FloatSlider, RadioButtons

# ----- Parámetros generales -----
frecuencias = np.arange(10, 501, 10)
fs = 5000  # Frecuencia de muestreo
duracion = 20  # en segundos
t = np.linspace(0, duracion, int(fs * duracion), endpoint=False)

# ----- Señal compuesta -----
senal = np.sum([np.sin(2 * np.pi * f * t) for f in frecuencias], axis=0)

# ----- Función para aplicar filtro, mostrar señal en el tiempo y espectro -----
def visualizar(f_low, f_high, t_inicio, t_fin, tipo_filtro):
    # Limitar el rango de tiempo
    idx_inicio = int(t_inicio * fs)
    idx_fin = int(t_fin * fs)

    # Crear filtro
    if tipo_filtro == 'Pasa banda':
        sos = butter(4, [f_low, f_high], btype='bandpass', fs=fs, output='sos')
        label = f'Filtro Pasa Banda: {f_low} - {f_high} Hz'
    elif tipo_filtro == 'Rechaza banda':
        sos = butter(4, [f_low, f_high], btype='bandstop', fs=fs, output='sos')
        label = f'Filtro Rechaza Banda: {f_low} - {f_high} Hz'

    # Aplicar filtro
    senal_filtrada = sosfilt(sos, senal)

    # ----------- Gráfica en el dominio del tiempo -----------
    plt.figure(figsize=(12, 4))
    plt.plot(t[idx_inicio:idx_fin], senal_filtrada[idx_inicio:idx_fin])
    plt.title(f"Señal filtrada en el tiempo ({label})")
    plt.xlabel("Tiempo [s]")
    plt.ylabel("Amplitud")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # ----------- Gráfica en el dominio de la frecuencia -----------
    n = len(senal_filtrada)
    fft_resultado = np.fft.fft(senal_filtrada)
    fft_frecuencias = np.fft.fftfreq(n, d=1/fs)
    magnitud = np.abs(fft_resultado) / n

    frecuencias_pos = fft_frecuencias[:n // 2]
    magnitud_pos = magnitud[:n // 2]

    plt.figure(figsize=(12, 4))
    markerline, stemlines, baseline = plt.stem(frecuencias_pos, magnitud_pos, basefmt=" ")
    baseline.set_visible(False)  # Oculta la línea base gruesa
    plt.title(f"Espectro de la señal filtrada ({label})")
    plt.xlabel("Frecuencia [Hz]")
    plt.ylabel("Magnitud")
    plt.xlim(0, 600)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# ----- Interfaz interactiva -----
interact(
    visualizar,
    f_low=IntSlider(value=100, min=10, max=490, step=10, description='F baja (Hz)'),
    f_high=IntSlider(value=300, min=20, max=500, step=10, description='F alta (Hz)'),
    t_inicio=FloatSlider(value=0.1, min=0, max=19.99, step=0.01, description='Inicio t (s)'),
    t_fin=FloatSlider(value=0.2, min=0.01, max=20.0, step=0.01, description='Fin t (s)'),
    tipo_filtro=RadioButtons(options=['Pasa banda', 'Rechaza banda'], description='Filtro')
)


interactive(children=(IntSlider(value=100, description='F baja (Hz)', max=490, min=10, step=10), IntSlider(val…

<function __main__.visualizar(f_low, f_high, t_inicio, t_fin, tipo_filtro)>